In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        # Adaptive Average Pooling to squeeze channel-wise statistics
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        # Excitation: two fully connected layers (1x1 convolutions)
        self.fc1 = nn.Conv2d(in_channels, in_channels // reduction, kernel_size=1, bias=False)
        self.fc2 = nn.Conv2d(in_channels // reduction, in_channels, kernel_size=1, bias=False)
        # Sigmoid activation to generate the attention weights
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Squeeze: global average pooling
        squeeze = self.avg_pool(x)
        # Excitation: pass through two fully connected layers (1x1 convolutions)
        excitation = self.fc1(squeeze)
        excitation = F.relu(excitation)
        excitation = self.fc2(excitation)
        # Apply sigmoid to scale the output between 0 and 1 (channel-wise attention)
        excitation = self.sigmoid(excitation)
        return x * excitation

In [ ]:
class ResidualBlockWithSE(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, reduction=16):
        super(ResidualBlockWithSE, self).__init__()
        # First convolutional layer
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        # Second convolutional layer
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        # Squeeze-and-Excitation block
        self.se_block = SEBlock(out_channels, reduction)
        # Shortcut connection to match input and output dimensions
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        # Perform the convolution operation
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        # Apply SE block
        out = self.se_block(out)
        # Add the shortcut connection
        out += self.shortcut(x)
        out = self.relu(out)
        return out